In [1]:
import random
import time
from collections import deque

class ResourceManager:
    def __init__(self, total_cpu, total_memory):
        self.total_cpu = total_cpu
        self.total_memory = total_memory
        self.available_cpu = total_cpu
        self.available_memory = total_memory
        self.processes = []

    def allocate(self, process):
        if self.available_cpu >= process.cpu_needed and self.available_memory >= process.memory_needed:
            self.available_cpu -= process.cpu_needed
            self.available_memory -= process.memory_needed
            self.processes.append(process)
            print(f"Allocated {process.cpu_needed}% CPU and {process.memory_needed}MB memory to {process.name}")
            return True
        return False

    def release(self, process):
        self.available_cpu += process.cpu_needed
        self.available_memory += process.memory_needed
        self.processes.remove(process)
        print(f"Released {process.cpu_needed}% CPU and {process.memory_needed}MB memory from {process.name}")

    def get_status(self):
        used_cpu = self.total_cpu - self.available_cpu
        used_memory = self.total_memory - self.available_memory
        print(f"Resource Status: {used_cpu}% CPU used, {used_memory}MB memory used")
        return used_cpu, used_memory

class Process:
    def __init__(self, name, cpu_needed, memory_needed):
        self.name = name
        self.cpu_needed = cpu_needed
        self.memory_needed = memory_needed
        self.running_time = random.randint(5, 15)
        self.retries = 0

    def run(self):
        print(f"{self.name} started, will run for {self.running_time} seconds.")
        time.sleep(self.running_time)

class Scheduler:
    def __init__(self, resource_manager, max_retries=5, max_iterations=50):
        self.resource_manager = resource_manager
        self.ready_queue = deque()
        self.waiting_queue = deque()  # Processes that couldn't be scheduled due to resource limitations
        self.max_retries = max_retries
        self.max_iterations = max_iterations

    def add_process(self, process):
        self.ready_queue.append(process)

    def schedule(self):
        iterations = 0
        while self.ready_queue or self.waiting_queue:
            iterations += 1

            if iterations > self.max_iterations:
                print("Maximum iterations reached, stopping scheduling.")
                break

            process = None
            if self.ready_queue:
                process = self.ready_queue.popleft()

            if process:
                if process.retries >= self.max_retries:
                    print(f"{process.name} has been retried {self.max_retries} times. Skipping.")
                    continue  # Skip this process if it has been retried too many times

                if self.resource_manager.allocate(process):
                    process.run()
                    self.resource_manager.release(process)
                else:
                    process.retries += 1
                    print(f"Insufficient resources to allocate to {process.name}, retrying... ({process.retries}/{self.max_retries})")
                    self.waiting_queue.append(process)

            if not process and self.waiting_queue:
                print("Waiting for resources to be available...")
                time.sleep(1)  # Sleep for a while to simulate waiting for resources

            self.process_waiting_queue()

    def process_waiting_queue(self):
        """Try to schedule processes from the waiting queue."""
        new_waiting_queue = deque()
        while self.waiting_queue:
            process = self.waiting_queue.popleft()
            if process.retries < self.max_retries:
                if self.resource_manager.allocate(process):
                    process.run()
                    self.resource_manager.release(process)
                else:
                    new_waiting_queue.append(process)
            else:
                print(f"{process.name} has been retried too many times and will be skipped.")
        self.waiting_queue = new_waiting_queue

    def monitor_resources(self):
        while True:
            self.resource_manager.get_status()
            time.sleep(2)

if __name__ == "__main__":
    total_cpu = 100
    total_memory = 2048
    resource_manager = ResourceManager(total_cpu, total_memory)

    scheduler = Scheduler(resource_manager)

    process_names = ["App1", "App2", "App3", "App4", "App5"]
    for name in process_names:
        cpu_needed = random.randint(10, 30)
        memory_needed = random.randint(100, 500)
        process = Process(name, cpu_needed, memory_needed)
        scheduler.add_process(process)

    import threading
    monitoring_thread = threading.Thread(target=scheduler.monitor_resources)
    monitoring_thread.daemon = True
    monitoring_thread.start()

    scheduler.schedule()


Resource Status: 0% CPU used, 0MB memory used
Allocated 26% CPU and 153MB memory to App1
App1 started, will run for 7 seconds.
Resource Status: 26% CPU used, 153MB memory used
Resource Status: 26% CPU used, 153MB memory used
Resource Status: 26% CPU used, 153MB memory used
Released 26% CPU and 153MB memory from App1
Allocated 16% CPU and 394MB memory to App2
App2 started, will run for 6 seconds.
Resource Status: 16% CPU used, 394MB memory used
Resource Status: 16% CPU used, 394MB memory used
Resource Status: 16% CPU used, 394MB memory used
Released 16% CPU and 394MB memory from App2
Allocated 15% CPU and 430MB memory to App3
App3 started, will run for 6 seconds.
Resource Status: 15% CPU used, 430MB memory used
Resource Status: 15% CPU used, 430MB memory used
Resource Status: 15% CPU used, 430MB memory used
Released 15% CPU and 430MB memory from App3
Allocated 28% CPU and 493MB memory to App4
App4 started, will run for 5 seconds.
Resource Status: 28% CPU used, 493MB memory used
Resource